# TG4 - Live Trading Dashboard
## Real-time Portfolio Monitoring & Signal Visualization

In [ ]:
import sys
sys.path.insert(0, '../src')

from data_fetcher import DataFetcher
from portfolio import Portfolio
from ensemble import Ensemble
from executor import Executor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import time
import yaml

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Initialize Portfolio & Data

In [ ]:
# Load config
with open('../config/exchanges.yaml') as f:
    config = yaml.safe_load(f)

portfolio = Portfolio(config['starting_balance'])
fetcher = DataFetcher()
executor = Executor(portfolio)

print(f"Starting Portfolio: {portfolio}")
print(f"Target allocation: BTC 40%, XRP 30%, RLUSD 20%, Stables 10%")

In [ ]:
# Fetch market data
symbols = ['XRP/USDT', 'BTC/USDT']
data = {}

for sym in symbols:
    print(f"Fetching {sym}...")
    df = fetcher.fetch_ohlcv('kraken', sym, '1h', 500)
    if not df.empty:
        data[sym] = df
        print(f"  {len(df)} candles loaded")

ensemble = Ensemble(data, portfolio)

## 2. Equity Curve Visualization

In [ ]:
def plot_equity_curve(portfolio_history):
    """Plot portfolio value over time"""
    if not portfolio_history:
        print("No history to plot")
        return
    
    timestamps = [h['timestamp'] for h in portfolio_history]
    values = [h['total_usd'] for h in portfolio_history]
    
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(timestamps, values, 'b-', linewidth=2)
    ax.axhline(y=values[0], color='r', linestyle='--', alpha=0.5, label='Starting Value')
    ax.fill_between(timestamps, values[0], values, alpha=0.3, 
                    where=[v >= values[0] for v in values], color='green')
    ax.fill_between(timestamps, values[0], values, alpha=0.3,
                    where=[v < values[0] for v in values], color='red')
    
    ax.set_title('Portfolio Equity Curve', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('Value (USD)')
    ax.legend()
    plt.tight_layout()
    plt.show()

# Generate some sample data
prices = {'USDT': 1.0, 'XRP': 2.2, 'BTC': 100000}
for _ in range(10):
    portfolio.record_snapshot(prices)
    
plot_equity_curve(portfolio.history)

## 3. Holdings Allocation

In [ ]:
def plot_holdings(portfolio, prices):
    """Plot current holdings as pie chart"""
    holdings = {}
    for asset, amount in portfolio.balances.items():
        if amount > 0 and not asset.endswith('_entry') and not asset.endswith('_lev'):
            usd_value = amount * prices.get(asset, 1.0)
            if usd_value > 1:  # Only show holdings > $1
                holdings[asset] = usd_value
    
    if not holdings:
        print("No holdings to display")
        return
    
    # Target allocation for comparison
    targets = {'BTC': 0.4, 'XRP': 0.3, 'RLUSD': 0.2, 'USDT': 0.05, 'USDC': 0.05}
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Current allocation
    colors = sns.color_palette('husl', len(holdings))
    axes[0].pie(holdings.values(), labels=holdings.keys(), autopct='%1.1f%%', colors=colors)
    axes[0].set_title('Current Allocation')
    
    # Bar comparison
    total = sum(holdings.values())
    current_pct = {k: v/total for k, v in holdings.items()}
    
    assets = list(set(list(current_pct.keys()) + list(targets.keys())))
    current_vals = [current_pct.get(a, 0) * 100 for a in assets]
    target_vals = [targets.get(a, 0) * 100 for a in assets]
    
    x = np.arange(len(assets))
    width = 0.35
    
    axes[1].bar(x - width/2, current_vals, width, label='Current', color='steelblue')
    axes[1].bar(x + width/2, target_vals, width, label='Target', color='coral', alpha=0.7)
    axes[1].set_ylabel('Allocation %')
    axes[1].set_title('Current vs Target Allocation')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(assets)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

plot_holdings(portfolio, prices)

## 4. Trade Log Visualization

In [ ]:
def plot_trade_log(trade_log):
    """Visualize trade history"""
    if not trade_log:
        print("No trades to display")
        return
    
    df = pd.DataFrame(trade_log)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Trade values over time
    colors = ['green' if s == 'buy' else 'red' for s in df['side']]
    axes[0].scatter(df['timestamp'], df['value'], c=colors, s=100, alpha=0.7)
    axes[0].set_title('Trade Values Over Time')
    axes[0].set_xlabel('Time')
    axes[0].set_ylabel('Trade Value (USD)')
    
    # Trade count by symbol
    symbol_counts = df['symbol'].value_counts()
    axes[1].bar(symbol_counts.index, symbol_counts.values, color='steelblue')
    axes[1].set_title('Trades by Symbol')
    axes[1].set_xlabel('Symbol')
    axes[1].set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()
    
    return df

# Display trade log if available
if executor.trade_log:
    trade_df = plot_trade_log(executor.trade_log)
    display(trade_df)
else:
    print("No trades executed yet")

## 5. Live Signal Monitor (Run Cell Repeatedly)

In [ ]:
def get_live_status():
    """Get current market status and signals"""
    prices = {'USDT': 1.0, 'USDC': 1.0, 'RLUSD': 1.0}
    
    for sym in ['XRP/USDT', 'BTC/USDT']:
        p = fetcher.get_best_price(sym)
        if p:
            base = sym.split('/')[0]
            prices[base] = list(p.values())[0]
    
    portfolio.record_snapshot(prices)
    signal = ensemble.get_signal('XRP/USDT')
    
    return prices, signal

# Get current status
prices, signal = get_live_status()

print(f"\n{'='*50}")
print(f"LIVE STATUS @ {time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*50}")
print(f"\nPrices:")
for asset, price in prices.items():
    if asset not in ['USDT', 'USDC', 'RLUSD']:
        print(f"  {asset}: ${price:,.2f}")

print(f"\nPortfolio Value: ${portfolio.get_total_usd(prices):,.2f}")
print(f"\nSignal: {signal['action']} (confidence: {signal['confidence']:.2f})")
print(f"  LSTM:     {signal['signals']['lstm']:.2f}")
print(f"  Arb:      {signal['signals']['arb']:.2f}")
print(f"  Rebalance: {signal['signals']['rebalance']:.2f}")

## 6. Continuous Dashboard (Auto-Refresh)

In [ ]:
# Run this cell for continuous monitoring (Ctrl+C to stop)
try:
    for i in range(10):  # 10 iterations, 30 sec apart
        clear_output(wait=True)
        
        prices, signal = get_live_status()
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Equity curve
        if portfolio.history:
            values = [h['total_usd'] for h in portfolio.history]
            axes[0, 0].plot(values, 'b-', linewidth=2)
            axes[0, 0].axhline(y=values[0], color='r', linestyle='--', alpha=0.5)
            axes[0, 0].set_title(f'Equity Curve (Current: ${values[-1]:,.2f})')
        
        # Holdings pie
        holdings = {k: v * prices.get(k, 1.0) for k, v in portfolio.balances.items() 
                   if v > 0 and not k.endswith('_entry')}
        if holdings:
            axes[0, 1].pie(holdings.values(), labels=holdings.keys(), autopct='%1.1f%%')
            axes[0, 1].set_title('Current Holdings')
        
        # Signal bars
        sig_names = list(signal['signals'].keys())
        sig_vals = list(signal['signals'].values())
        colors = ['green' if v > 0 else 'red' for v in sig_vals]
        axes[1, 0].bar(sig_names, sig_vals, color=colors)
        axes[1, 0].axhline(y=0, color='black', linewidth=0.5)
        axes[1, 0].set_title(f'Signal: {signal["action"]} ({signal["confidence"]:.2f})')
        axes[1, 0].set_ylim(-1, 1)
        
        # Price chart
        if 'XRP/USDT' in data:
            data['XRP/USDT']['close'].tail(100).plot(ax=axes[1, 1])
            axes[1, 1].set_title('XRP/USDT Price (Last 100 bars)')
        
        plt.suptitle(f'TG4 Live Dashboard - {time.strftime("%Y-%m-%d %H:%M:%S")}', fontsize=14)
        plt.tight_layout()
        plt.show()
        
        time.sleep(30)
        
except KeyboardInterrupt:
    print("Dashboard stopped")